<a target="_blank" href="https://colab.research.google.com/github/VectorInstitute/fed-rag/blob/main/docs/notebooks/rag_benchmarking_hf_mmlu.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

_(NOTE: if running on Colab, you will need to supply a WandB API Key in addition to your HFToken. Also, you'll need to change the runtime to a T4.)_

In [1]:
!uv pip install bitsandbytes -q

In [1]:
!uv pip install docker -q

# Basic Federated Fine-tuning of RAG Systems

## Knowledge Store

In [1]:
import docker
import os
import time

client = docker.from_env()
image_name = "vectorinstitute/qdrant-atlas-dec-wiki-2021:latest"

# first see if we need to pull the docker image
try:
    client.images.get(image_name)
    print(f"Image '{image_name}' already exists locally")
except docker.errors.ImageNotFound:
    print(f"Image '{image_name}' not found locally. Pulling...")
    # Pull with progress information
    for line in client.api.pull(image_name, stream=True, decode=True):
        if "progress" in line:
            print(f"\r{line['status']}: {line['progress']}", end="")
        elif "status" in line:
            print(f"\r{line['status']}", end="")
    print("\nPull complete!")

# run the Qdrant container
container = client.containers.run(
    "vectorinstitute/qdrant-atlas-dec-wiki-2021:latest",
    detach=True,  # -d flag
    name="tiny-wiki-dec2021-ks",  # --name
    ports={"6333/tcp": 6333, "6334/tcp": 6334},  # -p 6333:6333  # -p 6334:6334
    volumes={
        "qdrant_data": {  # -v qdrant_data:/qdrant_storage
            "bind": "/qdrant_storage",
            "mode": "rw",
        }
    },
    environment={"SAMPLE_SIZE": "tiny"},  # -e SAMPLE_SIZE=tiny
    device_requests=[
        docker.types.DeviceRequest(
            count=-1, capabilities=[["gpu"]]
        )  # --gpus all
    ],
    remove=False,  # Don't auto-remove when stopped
)

print(f"Container started with ID: {container.id}")

# wait a moment for the container to initialize
time.sleep(3)

# Check container status
container.reload()  # Refresh container data
print(f"Container status: {container.status}")
print(f"Container logs:")
print(container.logs().decode("utf-8"))

Image 'vectorinstitute/qdrant-atlas-dec-wiki-2021:latest' already exists locally
Container started with ID: f8e1721013ed18774e2c89629ac2b3d7f7c3ec6eab6ed48f73b99ee7308a1bc1
Container status: running
Container logs:
Starting Qdrant Atlas Knowledge Store container
Running database initialization check...
Using tiny sample mode...
Creating tiny sample file for testing...
Using tiny sample file: tiny-sample.jsonl
Verifying sample file creation...
✅ Sample file successfully created at: /app/data/atlas/enwiki-dec2021/tiny-sample.jsonl
File details:
-rw-r--r-- 1 root root 6785 Jun  8 01:32 /app/data/atlas/enwiki-dec2021/tiny-sample.jsonl
File content (first 3 lines):
{"id": "140", "title": "History of marine biology", "section": "James Cook", "text": " James Cook is well known for his voyages of exploration for the British Navy in which he mapped out a significant amount of the world's uncharted waters. Cook's explorations took him around the world twice and led to countless descriptions of p

In [2]:
print(f"Container status: {container.status}")

Container status: running


http://localhost:6333/dashboard#/collections

### Load the script to build the RAG System

In [3]:
import time

GIST_URL = f"https://gist.githubusercontent.com/nerdai/33e8445ab8b96783f34a7e0464e0b0f0/raw?fresh={int(time.time())}"

In [4]:
import requests

response = requests.get(GIST_URL)
rag_code = response.text

In [5]:
from IPython.display import Code, display

display(Code(rag_code, language="python"))

import torch
from transformers.generation.utils import GenerationConfig
from transformers.utils.quantization_config import BitsAndBytesConfig
from datasets import Dataset
from typing import Literal

from logging import INFO
from flwr.common.logger import log

from fed_rag import RAGSystem, RAGConfig
from fed_rag.knowledge_stores import QdrantKnowledgeStore
from fed_rag.retrievers import (
    HFSentenceTransformerRetriever,
)
from fed_rag.generators import HFPretrainedModelGenerator
from fed_rag.fl_tasks.huggingface import (
    HuggingFaceFlowerClient,
    HuggingFaceFlowerServer,
)
from fed_rag.data_structures import KnowledgeNode, NodeType
from fed_rag.trainers.huggingface.ralt import HuggingFaceTrainerForRALT
from fed_rag.trainer_managers.huggingface import HuggingFaceRAGTrainerManager

GRPC_MAX_MESSAGE_LENGTH = int(512 * 1024 * 1024 * 3.75)
PEFT_MODEL_NAME = "Styxxxx/llama2_7b_lora-quac"
BASE_MODEL_NAME = "meta-llama/Llama-2-7b-hf"
TRAIN_DATASET = Dataset.from_dict(
    # examples from Commonsense QA
    {
        "query": [
            "The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?",
            "Sammy wanted to go to where the people were.  Where might he go?",
            "To locate a choker not located in a jewelry box or boutique where would you go?",
            "Google Maps and other highway and street GPS services have replaced what?",
        ],
        "response": [
            "ignore",
            "populated areas",
            "jewelry store",
            "atlas",
        ],
    }
)
VAL_DATASET = Dataset.from_dict(
    {
        "query": [
            "The fox walked from the city into the forest, what was it looking for?"
        ],
        "response": [
            "natural habitat",
        ],
    }
)


def get_trainer_manager(server: bool) -> HuggingFaceRAGTrainerManager:
    # use the knowledge store in image: vectorinstitute/qdrant-atlas-dec-wiki-2021:latest
    knowledge_store = QdrantKnowledgeStore(
        collection_name="nthakur.dragon-plus-context-encoder",
        timeout=10,
    )
    retriever = HFSentenceTransformerRetriever(
        query_model_name="nthakur/dragon-plus-query-encoder",
        context_model_name="nthakur/dragon-plus-context-encoder",
        load_model_at_init=False,
    )

    # LLM generator
    generation_cfg = GenerationConfig(
        do_sample=True,
        eos_token_id=151643,
        bos_token_id=151643,
        max_new_tokens=2048,
        top_p=0.9,
        temperature=0.6,
        cache_implementation="offloaded",
        stop_strings="</response>",
    )
    if server:
        load_model_kwargs = {"device_map": "cpu", "torch_dtype": torch.float16}
    else:
        load_model_kwargs = {"device_map": "auto", "torch_dtype": torch.float16}
    generator = HFPretrainedModelGenerator(
        model_name="Qwen/Qwen2.5-0.5B",
        load_model_at_init=False,
        load_model_kwargs=load_model_kwargs,
        generation_config=generation_cfg,
    )

    # assemble rag system
    rag_config = RAGConfig(top_k=2)
    rag_system = RAGSystem(
        knowledge_store=knowledge_store,  # knowledge store loaded from knowledge_store.py
        generator=generator,
        retriever=retriever,
        rag_config=rag_config,
    )
    
    # the trainer object
    generator_trainer = HuggingFaceTrainerForRALT(
        rag_system=rag_system,
        train_dataset=TRAIN_DATASET,
    )
    # trainer manager object
    manager = HuggingFaceRAGTrainerManager(
        mode="generator",
        generator_trainer=generator_trainer,
    )
    return manager


def build_client(
    train_manager: HuggingFaceRAGTrainerManager,
) -> HuggingFaceFlowerClient:
    fl_task = train_manager.get_federated_task()
    model = train_manager.model
    log(INFO, f"loaded generator is on: {model.device}")
    return fl_task.client(
        model=model, train_dataset=TRAIN_DATASET, val_dataset=VAL_DATASET
    )


def build_server

### Federated Learning

In [6]:
import time

In [7]:
# write gist to script
with open("rag_federated_learning.py", "w") as f:
    f.write(rag_code)

In [8]:
from fed_rag.utils.notebook import ProcessMonitor

monitor = ProcessMonitor()

In [9]:
server_command = "python rag_federated_learning.py --component server"
client_command = "export CUDA_VISIBLE_DEVICES={client_id} && python rag_federated_learning.py --component client_{client_id}"

In [10]:
# start server process
monitor.start_process("server", server_command)

# give server time to standup
time.sleep(2)

✅ Started server (PID: 66185)


In [11]:
# start client processes
monitor.start_process("client_0", client_command.format(client_id="0"))
monitor.start_process("client_1", client_command.format(client_id="1"))

✅ Started client_0 (PID: 66214)
✅ Started client_1 (PID: 66217)


In [12]:
monitor.monitor_live(["server", "client_0", "client_1"])

🖥️  PROCESS MONITOR

server 🔴 STOPPED
------------------------------
[21:33:11] INFO :      Evaluation returned no results (`None`)
[21:33:11] INFO :
[21:33:11] INFO :      [ROUND 1]
[21:33:13] INFO :      configure_fit: strategy sampled 2 clients (out of 2)
[21:34:04] INFO :      aggregate_fit: received 2 results and 0 failures
[21:34:09] WARNING :   No fit_metrics_aggregation_fn provided
[21:34:09] INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
[21:34:24] INFO :      aggregate_evaluate: received 2 results and 0 failures
[21:34:24] WARNING :   No evaluate_metrics_aggregation_fn provided
[21:34:24] INFO :
[21:34:24] INFO :      [SUMMARY]
[21:34:24] INFO :      Run finished 1 round(s) in 73.33s
[21:34:24] INFO :      	History (loss, distributed):
[21:34:24] INFO :      		round 1: 0.41999998688697815
[21:34:24] INFO :

client_0 🔴 STOPPED
------------------------------
[21:33:55] 
[21:33:55] 
[21:33:55] 100%|██████████| 3/3 [00:26<00:00,  8.94s/it]
[21:33:55] /home/

In [13]:
monitor.stop_all()

🛑 Stopped server
🛑 Stopped client_0
🛑 Stopped client_1
🛑 All processes stopped


### Cleanup

In [14]:
# stop and remove container
container.stop()
container.remove()